In [2]:
%pip install -r requirements

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements'


In [3]:
import time
import requests
import sys

In [4]:
import requests
from bs4 import BeautifulSoup

def scrape_seattle_events(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        events = []
        event_containers = soup.find_all('div', class_='event-info')

        for event_container in event_containers:
            title = event_container.find('h2').text.strip()
            date = event_container.find('span', class_='date').text.strip()
            location = event_container.find('span', class_='location').text.strip()

            events.append({
                'title': title,
                'date': date,
                'location': location
            })

        return events

    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return []

url = "https://visitseattle.org/events"
seattle_events = scrape_seattle_events(url)
print("URL : ", url)

print(sys.getsizeof(seattle_events))

# Access the first element
first_element = seattle_events[2]

print(f"The first element in the list is: {first_element}")


# Print the first few events
for event in seattle_events[:5]:
  print(f"Title: {event['title']}\nDate: {event['date']}\nLocation: {event['location']}\n---")

URL :  https://visitseattle.org/events
56


IndexError: list index out of range

In [5]:
from bs4 import BeautifulSoup
url = "https://visitseattle.org/events/page/1"

# Create -> POST
# Read   -> GET
# Update -> PUT
# Delete -> DELETE
#
# requests.post("https://concert.com/buy")
# requests.put("https://github.com/ianchen06")
# requests.delete("https://facebook.com/ianchen06/post/1")

res = requests.get(url)
res.status_code
res.text

# Specify the file name
html_file_name = "seattleevents.html"

# Write the HTML content to the file
with open(html_file_name, "w", encoding="utf-8") as html_file:
    html_file.write(res.text)
soup = BeautifulSoup(res.text, "html.parser")

selector = "div.search-result-preview > div > h3 > a"

a_eles = soup.select(selector)
a_eles
[x['href'] for x in a_eles]


['https://visitseattle.org/events/steinunn-porarinsdottir-wayfinders/',
 'https://visitseattle.org/events/hanako-oleary-izanami/',
 'https://visitseattle.org/events/gage-alumni-show/',
 'https://visitseattle.org/events/genre-box/',
 'https://visitseattle.org/events/bohemia/',
 'https://visitseattle.org/events/alice-in-shadowland/',
 'https://visitseattle.org/events/lee-ritenour-and-dave-grusin/',
 'https://visitseattle.org/events/tacoma-home-garden-show/',
 'https://visitseattle.org/events/annie-eastwood-with-kimball-and-the-fugitives/']

In [6]:
from bs4 import BeautifulSoup
import requests

# Define the base URL
base_url = "https://visitseattle.org/events/page/"

# Iterate through pages 1 to 9
for page_num in range(1, 10):
    url = f"{base_url}{page_num}"

    # Send an HTTP request to the URL
    res = requests.get(url)

    # Check if the request was successful (status code 200)
    if res.status_code == 200:
        # Specify the file name
        html_file_name = f"seattleevents_page_{page_num}.html"

        # Write the HTML content to the file
        with open(html_file_name, "w", encoding="utf-8") as html_file:
            html_file.write(res.text)

        # Parse the HTML content
        soup = BeautifulSoup(res.text, "html.parser")

        # Extract event links using the specified selector
        selector = "div.search-result-preview > div > h3 > a"
        a_eles = soup.select(selector)

        # Print event links
        event_links = [x['href'] for x in a_eles]
        print(f"Event links on page {page_num}: {event_links}")

    else:
        print(f"Failed to retrieve the webpage. Status code: {res.status_code}")
        
        

Event links on page 1: ['https://visitseattle.org/events/steinunn-porarinsdottir-wayfinders/', 'https://visitseattle.org/events/hanako-oleary-izanami/', 'https://visitseattle.org/events/gage-alumni-show/', 'https://visitseattle.org/events/genre-box/', 'https://visitseattle.org/events/bohemia/', 'https://visitseattle.org/events/alice-in-shadowland/', 'https://visitseattle.org/events/lee-ritenour-and-dave-grusin/', 'https://visitseattle.org/events/tacoma-home-garden-show/', 'https://visitseattle.org/events/annie-eastwood-with-kimball-and-the-fugitives/']
Event links on page 2: ['https://visitseattle.org/events/black-and-boujee-panel-discussion/', 'https://visitseattle.org/events/celebrate-asia/', 'https://visitseattle.org/events/djangos-birthday-concert-with-ranger-and-the-re-arrangers/', 'https://visitseattle.org/events/family-day-with-birds-connect-seattle/', 'https://visitseattle.org/events/fossil-finders-ice-ages/', 'https://visitseattle.org/events/seattle-kraken-vs-columbus-blue-jac

In [7]:

from bs4 import BeautifulSoup
import requests

# Global list to store all event links across pages
global_event_list = []

def extract(page_num):
    global global_event_list

    # Define the base URL
    base_url = "https://visitseattle.org/events/page/"

    # Construct the URL for the given page
    url = f"{base_url}{page_num}"

    # Send an HTTP request to the URL
    res = requests.get(url)

    # Check if the request was successful (status code 200)
    if res.status_code == 200:
        # Specify the file name
        html_file_name = f"seattleevents_page_{page_num}.html"

        # Write the HTML content to the file
        with open(html_file_name, "w", encoding="utf-8") as html_file:
            html_file.write(res.text)

        # Parse the HTML content
        soup = BeautifulSoup(res.text, "html.parser")

        # Extract event links using the specified selector
        selector = "div.search-result-preview > div > h3 > a"
        a_eles = soup.select(selector)

        # Store event links in the global list
        event_links = [x['href'] for x in a_eles]
        global_event_list.extend(event_links)

        # Print event links
        print(f"Event links on page {page_num}: {event_links}")

        # Call the function recursively for the next page (if applicable)
        if page_num < 9:
            extract(page_num + 1)

    else:
        print(f"Failed to retrieve the webpage. Status code: {res.status_code}")

# Start the extraction process from page 1
extract(1)

# Print the global list of all event links
print("Global Event List:", global_event_list)

Event links on page 1: ['https://visitseattle.org/events/steinunn-porarinsdottir-wayfinders/', 'https://visitseattle.org/events/hanako-oleary-izanami/', 'https://visitseattle.org/events/gage-alumni-show/', 'https://visitseattle.org/events/genre-box/', 'https://visitseattle.org/events/bohemia/', 'https://visitseattle.org/events/alice-in-shadowland/', 'https://visitseattle.org/events/lee-ritenour-and-dave-grusin/', 'https://visitseattle.org/events/tacoma-home-garden-show/', 'https://visitseattle.org/events/annie-eastwood-with-kimball-and-the-fugitives/']
Event links on page 2: ['https://visitseattle.org/events/black-and-boujee-panel-discussion/', 'https://visitseattle.org/events/celebrate-asia/', 'https://visitseattle.org/events/djangos-birthday-concert-with-ranger-and-the-re-arrangers/', 'https://visitseattle.org/events/family-day-with-birds-connect-seattle/', 'https://visitseattle.org/events/fossil-finders-ice-ages/', 'https://visitseattle.org/events/seattle-kraken-vs-columbus-blue-jac

In [19]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_event_details(event_url):
    # Send an HTTP GET request to the detail page URL
    res = requests.get(event_url)
    print(event_url)

    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'html.parser')

        # Extract event details
        #name_element = soup.find('h1', class_='page-title')
        #date_element = soup.find('span', class_='span')
        #location_element = soup.find('span', class_='location')
        #event_type_element = soup.find('span', class_='event-type')
        #region_element = soup.find('span', class_='region')
        
        name = soup.find('h1', class_='page-title').text.strip()
        date = soup.find("h4").find_all("span")[0].text.strip()
        location = soup.find("h4").find_all("span")[1].text.strip()
        event_type = soup.find_all("a", class_="button big medium black category")[0].text.strip()
        region = soup.find_all("a", class_="button big medium black category")[1].text.strip()

        # Check if elements are found before accessing their properties
        #name = name_element.text.strip() if name_element else "N/A"
        #date = date_element.text.strip() if date_element else "N/A"
        #location = location_element.text.strip() if location_element else "N/A"
        #event_type = event_type_element.text.strip() if event_type_element else "N/A"
        #region = region_element.text.strip() if region_element else "N/A"

        return {
            'Name': name,
            'Date': date,
            'Location': location,
            'Type': event_type,
            'Region': region
        }

    else:
        print(f"Failed to retrieve the event details. Status code: {res.status_code}")
        return None

# List of event URLs obtained in step 1
event_urls = global_event_list

# Create a CSV file to store the event details
csv_file_path = "events.csv"

# Open the CSV file in write mode with a CSV writer
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['Name', 'Date', 'Location', 'Type', 'Region']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write the header row to the CSV file
    writer.writeheader()

    # Loop through each event URL and scrape details
    for event_url in event_urls:
        event_details = scrape_event_details(event_url)

        if event_details:
            # Write event details to the CSV file
            writer.writerow(event_details)

print(f"Event details have been scraped and stored in '{csv_file_path}'.")

https://visitseattle.org/events/steinunn-porarinsdottir-wayfinders/
https://visitseattle.org/events/hanako-oleary-izanami/
https://visitseattle.org/events/gage-alumni-show/
https://visitseattle.org/events/genre-box/
https://visitseattle.org/events/bohemia/
https://visitseattle.org/events/alice-in-shadowland/
https://visitseattle.org/events/lee-ritenour-and-dave-grusin/
https://visitseattle.org/events/tacoma-home-garden-show/
https://visitseattle.org/events/annie-eastwood-with-kimball-and-the-fugitives/
https://visitseattle.org/events/black-and-boujee-panel-discussion/
https://visitseattle.org/events/celebrate-asia/
https://visitseattle.org/events/djangos-birthday-concert-with-ranger-and-the-re-arrangers/
https://visitseattle.org/events/family-day-with-birds-connect-seattle/
https://visitseattle.org/events/fossil-finders-ice-ages/
https://visitseattle.org/events/seattle-kraken-vs-columbus-blue-jackets/
https://visitseattle.org/events/tacoma-stars-vs-san-diego-sockers-3/
https://visitsea

In [8]:
from bs4 import BeautifulSoup
import requests
import csv

# Global list to store all event links across pages
global_event_list = []

def extract(page_num):
    global global_event_list

    # Define the base URL
    base_url = "https://visitseattle.org/events/page/"

    # Construct the URL for the given page
    url = f"{base_url}{page_num}"

    # Send an HTTP request to the URL
    res = requests.get(url)

    # Check if the request was successful (status code 200)
    if res.status_code == 200:
        # Specify the file name
        html_file_name = f"seattleevents_page_{page_num}.html"

        # Write the HTML content to the file
        with open(html_file_name, "w", encoding="utf-8") as html_file:
            html_file.write(res.text)

        # Parse the HTML content
        soup = BeautifulSoup(res.text, "html.parser")

        # Extract event links using the specified selector
        selector = "div.search-result-preview > div > h3 > a"
        a_eles = soup.select(selector)

        # Store event links in the global list
        event_links = [x['href'] for x in a_eles]
        global_event_list.extend(event_links)

        # Print event links
        print(f"Event links on page {page_num}: {event_links}")

        # Call the function recursively for the next page (if applicable)
        if page_num < 9:
            extract(page_num + 1)

    else:
        print(f"Failed to retrieve the webpage. Status code: {res.status_code}")

# Function to look up the location using OpenStreetMap Nominatim API
def lookup_location(location):
    nominatim_url = "https://nominatim.openstreetmap.org/search"
    params = {'q': location, 'format': 'json'}

    response = requests.get(nominatim_url, params=params)

    if response.status_code == 200:
        data = response.json()
        if data:
            return {
                'latitude': data[0]['lat'],
                'longitude': data[0]['lon'],
                'display_name': data[0]['display_name']
            }
    else:
        print(f"Failed to look up the location. Status code: {response.status_code}")
    return None

# Function to fetch weather data using National Weather Service API
def fetch_weather(latitude, longitude):
    weather_api_url = f"https://api.weather.gov/points/{latitude},{longitude}/forecast"
    
    response = requests.get(weather_api_url)

    if response.status_code == 200:
        data = response.json()
        if 'properties' in data:
            periods = data['properties']['periods']
            day_time_forecast = next((period for period in periods if 'Day' in period['name']), None)

            if day_time_forecast:
                return {
                    'temperature': day_time_forecast['temperature'],
                    'short_forecast': day_time_forecast['shortForecast']
                }
    else:
        print(f"Failed to fetch weather data. Status code: {response.status_code}")
    return None

# Start the extraction process from page 1
extract(1)

# Create a CSV file to store the data
csv_file_path = "events_data.csv"

# Open the CSV file in write mode with a CSV writer
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['Event Link', 'Location', 'Latitude', 'Longitude', 'Weather Temperature', 'Weather Forecast']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write the header row to the CSV file
    writer.writeheader()

    # Loop through each event link, look up the location, fetch weather, and write to CSV
    for event_link in global_event_list:
        # Look up the location
        location_data = lookup_location(event_link)

        if location_data:
            # Fetch weather data
            weather_data = fetch_weather(location_data['latitude'], location_data['longitude'])

            # Write data to CSV
            writer.writerow({
                'Event Link': event_link,
                'Location': location_data['display_name'],
                'Latitude': location_data['latitude'],
                'Longitude': location_data['longitude'],
                'Weather Temperature': weather_data['temperature'] if weather_data else "N/A",
                'Weather Forecast': weather_data['short_forecast'] if weather_data else "N/A"
            })

print(f"Data has been scraped, and weather/location information stored in '{csv_file_path}'.")

Event links on page 1: ['https://visitseattle.org/events/steinunn-porarinsdottir-wayfinders/', 'https://visitseattle.org/events/hanako-oleary-izanami/', 'https://visitseattle.org/events/gage-alumni-show/', 'https://visitseattle.org/events/genre-box/', 'https://visitseattle.org/events/bohemia/', 'https://visitseattle.org/events/alice-in-shadowland/', 'https://visitseattle.org/events/lee-ritenour-and-dave-grusin/', 'https://visitseattle.org/events/tacoma-home-garden-show/', 'https://visitseattle.org/events/annie-eastwood-with-kimball-and-the-fugitives/']
Event links on page 2: ['https://visitseattle.org/events/black-and-boujee-panel-discussion/', 'https://visitseattle.org/events/celebrate-asia/', 'https://visitseattle.org/events/djangos-birthday-concert-with-ranger-and-the-re-arrangers/', 'https://visitseattle.org/events/family-day-with-birds-connect-seattle/', 'https://visitseattle.org/events/fossil-finders-ice-ages/', 'https://visitseattle.org/events/seattle-kraken-vs-columbus-blue-jac

In [9]:
# location name to lat lon

# Option 1: just string manipulation
base_url = "https://nominatim.openstreetmap.org/search.php"
query_params_str = "?q=Wallingford%2C+Seattle&format=jsonv2"
ful_url = base_url + query_params_str

# Option 2: use dictionary to represent query params
#           use requests.get(url, params=query_params) to attach the query param dict
query_params = {
    "q": "Wallingford, Seattle",
    "format": "jsonv2"
}

res = requests.get(base_url, params=query_params)
res.json()

weather_api_url = f"https://api.weather.gov/points/{res.json()[0]['lat']},{res.json()[0]['lon']}"

print(weather_api_url)

res = requests.get(weather_api_url)

print(res)

point_dict = res.json()

forcast_url = point_dict['properties']['forecast']

res_forecast = requests.get(forcast_url)
weather_data = res_forecast.json()
print(res_forecast.json().keys())

point_dict= res_forecast.json()
forecast = weather_data['properties']['periods'][0]['detailedForecast']

#forcast_url = point_dict['properties']['detailedForecast']
print(forecast)



"""
data = res.json()

def lookup_location(latitude, longitude):
    if data:
        return {
                'latitude': data[0]['lat'],
                'longitude': data[0]['lon'],
                'display_name': data[0]['display_name']
                }
    else:
        print(f"Failed to look up the location. Status code: {response.status_code}")
        return None

location_data = lookup_location(event_link)
print(location_data['latitude'])
print(location_data['longitude'])

#weather_api_url = f"https://api.weather.gov/points/{location_data['latitude']},{location_data['longitude}/forecast"
    
res = requests.get(weather_api_url)

print(res)
"""


https://api.weather.gov/points/47.6594631,-122.3343417
<Response [200]>
dict_keys(['@context', 'type', 'geometry', 'properties'])
Rain before midnight, then rain and patchy fog. Cloudy, with a low around 52. South wind around 9 mph. Chance of precipitation is 90%. New rainfall amounts between a tenth and quarter of an inch possible.


'\ndata = res.json()\n\ndef lookup_location(latitude, longitude):\n    if data:\n        return {\n                \'latitude\': data[0][\'lat\'],\n                \'longitude\': data[0][\'lon\'],\n                \'display_name\': data[0][\'display_name\']\n                }\n    else:\n        print(f"Failed to look up the location. Status code: {response.status_code}")\n        return None\n\nlocation_data = lookup_location(event_link)\nprint(location_data[\'latitude\'])\nprint(location_data[\'longitude\'])\n\n#weather_api_url = f"https://api.weather.gov/points/{location_data[\'latitude\']},{location_data[\'longitude}/forecast"\n    \nres = requests.get(weather_api_url)\n\nprint(res)\n'